In [1]:
!git clone https://github.com/lsewcx/Pytorch-UNet.git

Cloning into 'Pytorch-UNet'...
remote: Enumerating objects: 18387, done.
remote: Counting objects: 100% (8534/8534), done.
remote: Compressing objects: 100% (378/378), done.
remote: Total 18387 (delta 8198), reused 8490 (delta 8156), pack-reused 9853 (from 1)
Receiving objects: 100% (18387/18387), 190.27 MiB | 40.38 MiB/s, done.
Resolving deltas: 100% (13201/13201), done.
Updating files: 100% (2558/2558), done.


In [8]:
!pip install segmentation_models_pytorch
!pip install -U albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.6/214.6 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.13
    Uninstalling albucore-0.0.13:
      Successfully uninstalled albucore-0.0.13
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.14
    Uninstalling albumentations-1.4.14:
      Successfully uninstalled albumentations-1.4.14


In [3]:
%cd Pytorch-UNet
!rm -rf NEU_Seg-main
!unzip -q ceping/NEU_Seg-main.zip

/kaggle/working/Pytorch-UNet


In [4]:
!mv /kaggle/working/Pytorch-UNet/NEU_Seg-main/images/test/* /kaggle/working/Pytorch-UNet/NEU_Seg-main/images/training/
!mv /kaggle/working/Pytorch-UNet/NEU_Seg-main/annotations/test/* /kaggle/working/Pytorch-UNet/NEU_Seg-main/annotations/training/

In [ ]:
%cd /kaggle/working/Pytorch-UNet
!git pull
!rm -rf best_model.pth best_model.pt
!python unet_train.py --classes 4 --batch-size 32 --epochs 100 --scale 1 --model UNet --v 10

/kaggle/working/Pytorch-UNet
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 554 bytes | 554.00 KiB/s, done.
From https://github.com/lsewcx/Pytorch-UNet
   fff6a77e..37f9d9ad  master     -> origin/master
Updating fff6a77e..37f9d9ad
Fast-forward
 utils/data_loading.py | 28 +++++++++++++++++-----------
 1 file changed, 17 insertions(+), 11 deletions(-)
INFO: Using device cuda
INFO: 模型的参数量: 67.16M
INFO: Network: Unet
INFO: Creating dataset with 4170 examples
INFO: Scanning mask files to determine unique values
  0%|                                                  | 0/4170 [00:00<?, ?it/s]
INFO: Creating dataset with 4170 examples
INFO: Scanning mask files to determine unique values
100%|██████████████████████████████████████| 4170/4170 [00:11<00:00, 351.44it/s]
INFO: Unique mask values: [0, 1, 2, 3]
wandb: Cu

In [ ]:
%cd /kaggle/working/Pytorch-UNet
!git pull
!python unet_predict.py --model best_model.pth --scale 1 --model-name UNet -i NEU_Seg-main/images/test --batch-size 32

In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
%cd Pytorch-UNet
!git pull

In [ ]:
%cd Pytorch-UNet
!rm -rf best_model.pth best_model.pt
!python train.py --classes 4 --batch-size 16 --epochs 100 --scale 1 --model selfnet --v 20

In [ ]:
import torch
import segmentation_models_pytorch as smp

model = torch.load('/kaggle/working/Pytorch-UNet/ceping/best_model.pth')

print(model)

In [ ]:
# !rm -rf archive_name.zip /kaggle/working/Pytorch-UNet/test_predictions 
# %cd /kaggle/working/Pytorch-UNet
# !mv best_model.pth ceping/best_model.pth
# !python predict.py --model  /kaggle/working/Pytorch-UNet/ceping/best_model.pth --scale 1 --model-name selfnet -i NEU_Seg-main/images/test
!rm -rf ceping/test_predictions
!mv test_predictions ceping/test_predictions

In [ ]:
!rm -rf ceping/baseline_predictions
!mv test_predictions ceping/baseline_predictions

In [ ]:
%cd ceping
!python main.py
%cd ..

In [ ]:
%cd /kaggle/working/Pytorch-UNet
!rm -rf submsion.zip
!zip submsion.zip ceping/best_model.pth ceping/results.json  ceping/results.txt ceping/test_predictions/*

In [ ]:
%cd /kaggle/working
!rm -rf predictions.zip
!zip predictions.zip /kaggle/working/Pytorch-UNet/test_predictions/*

In [ ]:
import os
os.chdir('/kaggle/working')
print(os.getcwd())
print(os.listdir("/kaggle/working"))
from IPython.display import FileLink
FileLink("predictions.zip")
# FileLink('Pytorch-UNet/submsion.zip')

In [ ]:
!pip install segmentation-models-pytorch